In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from plotly import express as px
from plotly import graph_objects as go

from tensorflow.keras.regularizers import L2
from numpy import pi
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.optimizers import Adam

color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]


In [2]:
data = make_circles(n_samples=1000,noise=0.1,factor=0.5)

In [3]:
x = data[0]
y = data[1]

In [4]:
fig = go.Figure(data=[
                      go.Scatter(
                          x = x[:,0],
                          y = x[:,1],
                          mode = "markers",
                          marker = dict(
                              color=y,
                              colorscale=color_chart[0:2]
                          ),
                      )
])

fig.show()

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x,y)

In [6]:
train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
val = tf.data.Dataset.from_tensor_slices((x_val,y_val))

train_batch = train.shuffle(len(train)).batch(8)
val_batch = val.shuffle(len(val)).batch(8)

In [7]:
model = Sequential([
                    Dense(4,"relu", input_shape=[2]),
                    Dense(1,"sigmoid")                                    
])
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
model.compile(optimizer=Adam(0.01),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

In [9]:
history = model.fit(train_batch, epochs=100, validation_data=val_batch)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6771 - binary_accuracy: 0.5480 - val_loss: 0.6333 - val_binary_accuracy: 0.7440
Epoch 2/100
94/94 [==============================] - 0s 952us/step - loss: 0.6146 - binary_accuracy: 0.7227 - val_loss: 0.5600 - val_binary_accuracy: 0.6760
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5552 - binary_accuracy: 0.7387 - val_loss: 0.4772 - val_binary_accuracy: 0.8360
Epoch 4/100
94/94 [==============================] - 0s 930us/step - loss: 0.4609 - binary_accuracy: 0.8773 - val_loss: 0.3704 - val_binary_accuracy: 0.9640
Epoch 5/100
94/94 [==============================] - 0s 930us/step - loss: 0.3437 - binary_accuracy: 0.9493 - val_loss: 0.2710 - val_binary_accuracy: 0.9880
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.2560 - binary_accuracy: 0.9760 - val_loss: 0.2145 - val_binary_accuracy: 0.9800
Epoch 7/100
94/94 [==============================] - 0s 1ms/step

In [10]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

In [11]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["binary_accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[4]
                          )),
                      go.Scatter(
                          y=history.history["val_binary_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[5]
                          ))
])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'    
)
fig.show()

In [12]:
x1_example = np.linspace(-2,2,100)
x2_example = np.linspace(-2,2,100)

x1, x2 = np.meshgrid(x1_example,x2_example)

example = np.array([[xx1,xx2] for xx1,xx2 in zip(x1.reshape(-1),x2.reshape(-1))])

pred_example = model(example).numpy().reshape([100,100])

fig = go.Figure(data=[
    go.Contour(
        x=x1_example,
        y=x2_example,
        z=pred_example,
        colorscale=[color_chart[-2],color_chart[-1]]
    )
])
fig.add_trace(go.Scatter(x = x[:,0],
                         y = x[:,1],
                         mode = "markers",
                         marker = dict(
                             color=y,
                             colorscale=color_chart[0:2])
                         )
)
fig.show()

In [13]:
N = 1000
theta = np.sqrt(np.random.rand(N))*4*pi # np.linspace(0,2*pi,100)

r_a = 2*theta + pi
data_a = np.array([np.cos(theta)*r_a, np.sin(theta)*r_a]).T
x_a = data_a + np.random.randn(N,2)
x_a = x_a/np.abs(x_a).max()

r_b = -2*theta - pi
data_b = np.array([np.cos(theta)*r_b, np.sin(theta)*r_b]).T
x_b = data_b + np.random.randn(N,2)
x_b = x_b/np.abs(x_b).max()

res_a = np.append(x_a, np.zeros((N,1)), axis=1)
res_b = np.append(x_b, np.ones((N,1)), axis=1)

res = np.append(res_a, res_b, axis=0)
np.random.shuffle(res)

np.savetxt("result.csv", res, delimiter=",", header="x,y,label", comments="", fmt='%.5f')

fig = go.Figure(data=[
                      go.Scatter(x=res[:,0],
                                 y=res[:,1],
                                 mode="markers",
                                 marker=dict(
                                     color=res[:,2],
                                     colorscale=color_chart[0:2]
                                     )
                                 )
                      ]
                )
fig.show()

data = res[:,0:2]
target = res[:,-1]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(data, target)

In [15]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(len(X_train)).batch(8)
val_batch = tf.data.Dataset.from_tensor_slices((X_val,y_val)).shuffle(len(X_val)).batch(8)

In [16]:
next(iter(train_batch))

(<tf.Tensor: shape=(8, 2), dtype=float64, numpy=
 array([[ 0.3749894 ,  0.34345134],
        [-0.09144092, -0.83111831],
        [ 0.27724452,  0.75804177],
        [-0.75499902,  0.17182734],
        [-0.39117476,  0.76228791],
        [-0.40104904,  0.57535884],
        [-0.03794192,  0.69461453],
        [ 0.36598124, -0.79836505]])>,
 <tf.Tensor: shape=(8,), dtype=float64, numpy=array([0., 0., 1., 0., 1., 0., 0., 0.])>)

In [17]:
model = Sequential([
                    Dense(64,"relu",input_shape=[2]),
                    Dense(32,"relu"),
                    Dense(1,"sigmoid")
])

In [18]:
model.compile(optimizer=Adam(0.01),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

In [19]:
history = model.fit(train_batch,epochs=100,validation_data=val_batch)

Epoch 1/100
188/188 [==============================] - 1s 1ms/step - loss: 0.6639 - binary_accuracy: 0.5907 - val_loss: 0.6369 - val_binary_accuracy: 0.5800
Epoch 2/100
188/188 [==============================] - 0s 2ms/step - loss: 0.6166 - binary_accuracy: 0.6073 - val_loss: 0.5368 - val_binary_accuracy: 0.6160
Epoch 3/100
188/188 [==============================] - 0s 1ms/step - loss: 0.5377 - binary_accuracy: 0.6433 - val_loss: 0.4616 - val_binary_accuracy: 0.6700
Epoch 4/100
188/188 [==============================] - 0s 878us/step - loss: 0.4559 - binary_accuracy: 0.7627 - val_loss: 0.3511 - val_binary_accuracy: 0.8720
Epoch 5/100
188/188 [==============================] - 0s 906us/step - loss: 0.3170 - binary_accuracy: 0.8540 - val_loss: 0.2389 - val_binary_accuracy: 0.9000
Epoch 6/100
188/188 [==============================] - 0s 888us/step - loss: 0.2810 - binary_accuracy: 0.8860 - val_loss: 0.1851 - val_binary_accuracy: 0.9300
Epoch 7/100
188/188 [==============================]

In [20]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

In [21]:
x1_example = np.linspace(-1,1,100)
x2_example = np.linspace(-1,1,100)

x1, x2 = np.meshgrid(x1_example,x2_example)

example = np.array([[xx1,xx2] for xx1,xx2 in zip(x1.reshape(-1),x2.reshape(-1))])

pred_example = model(example).numpy().reshape([100,100])

fig = go.Figure(data=[
    go.Contour(
        x=x1_example,
        y=x2_example,
        z=pred_example,
        colorscale=[color_chart[-2],color_chart[-1]]
    )
])
fig.add_trace(go.Scatter(x = data[:,0],
                         y = data[:,1],
                         mode = "markers",
                         marker = dict(
                             color=target,
                             colorscale=color_chart[0:2])
                         )
)
fig.show()

In [22]:
l2 = L2(0.0001)

model = Sequential([
                    Dense(64,"relu",input_shape=[2], kernel_regularizer=l2),
                    Dense(32,"relu", kernel_regularizer=l2),
                    Dense(1,"sigmoid", kernel_regularizer=l2)
])

model.compile(optimizer=Adam(0.01),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])


In [23]:
history = model.fit(train_batch,epochs=500,validation_data=val_batch)

Epoch 1/500
188/188 [==============================] - 1s 1ms/step - loss: 0.6646 - binary_accuracy: 0.5860 - val_loss: 0.6538 - val_binary_accuracy: 0.5880
Epoch 2/500
188/188 [==============================] - 0s 885us/step - loss: 0.6430 - binary_accuracy: 0.5873 - val_loss: 0.5972 - val_binary_accuracy: 0.6140
Epoch 3/500
188/188 [==============================] - 0s 915us/step - loss: 0.5604 - binary_accuracy: 0.6433 - val_loss: 0.5180 - val_binary_accuracy: 0.6140
Epoch 4/500
188/188 [==============================] - 0s 891us/step - loss: 0.5087 - binary_accuracy: 0.6627 - val_loss: 0.5290 - val_binary_accuracy: 0.6540
Epoch 5/500
188/188 [==============================] - 0s 896us/step - loss: 0.4838 - binary_accuracy: 0.6887 - val_loss: 0.4379 - val_binary_accuracy: 0.6800
Epoch 6/500
188/188 [==============================] - 0s 915us/step - loss: 0.4747 - binary_accuracy: 0.6887 - val_loss: 0.4652 - val_binary_accuracy: 0.6960
Epoch 7/500
188/188 [===========================

In [24]:
x1_example = np.linspace(-1,1,100)
x2_example = np.linspace(-1,1,100)

x1, x2 = np.meshgrid(x1_example,x2_example)

example = np.array([[xx1,xx2] for xx1,xx2 in zip(x1.reshape(-1),x2.reshape(-1))])

pred_example = model(example).numpy().reshape([100,100])

fig = go.Figure(data=[
    go.Contour(
        x=x1_example,
        y=x2_example,
        z=pred_example,
        colorscale=[color_chart[-2],color_chart[-1]]
    )
])
fig.add_trace(go.Scatter(x = data[:,0],
                         y = data[:,1],
                         mode = "markers",
                         marker = dict(
                             color=target,
                             colorscale=color_chart[0:2])
                         )
)
fig.show()